In [107]:
import requests
from urllib.parse import urlencode
import time
import pandas as pd
from datetime import datetime, timedelta

CLIENT_ID = '694d0e01cb0241cc956c7207f715900b'
CLIENT_SECRET = '99dfe1b42eb04d6ab0d85e00d5a7083f'
REDIRECT_URI = 'http://localhost:8888/?token=54e55c4d8d01005ed79cb476196a539a3bc83783e18d502d'
SCOPE = 'user-top-read'

def get_user_authorization():
    auth_url = 'https://accounts.spotify.com/authorize'
    payload = {
        'client_id': CLIENT_ID,
        'response_type': 'code',
        'redirect_uri': REDIRECT_URI,
        'scope': SCOPE
    }
    url = f"{auth_url}?{urlencode(payload)}"
    print(f"Go to the following URL to authorize the application: {url}")

def get_access_token(auth_code):
    token_url = 'https://accounts.spotify.com/api/token'
    payload = {
        'grant_type': 'authorization_code',
        'code': auth_code,
        'redirect_uri': REDIRECT_URI,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET
    }
    response = requests.post(token_url, data=payload)
    response_data = response.json()
    return response_data['access_token']

In [108]:
get_user_authorization()

Go to the following URL to authorize the application: https://accounts.spotify.com/authorize?client_id=694d0e01cb0241cc956c7207f715900b&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2F%3Ftoken%3D54e55c4d8d01005ed79cb476196a539a3bc83783e18d502d&scope=user-top-read


In [109]:
auth_code = "AQAm_KwZM9Ji9xdqZs2DHLdvB7HzhLajGl8il0fTVWwk5dQ_bJSjXi2N82MmDYywcuOMRODKjWkOYJjQ7ClnJck2y0-K1x9TkF3lH1urm-WnS5hukzO1FgUUxPdMGc4PUCMLlVVKjbtvl2Jh8C2F6jVnzfW_M8jXmUPoY1XzBgE4E4uKRL0Notr9sgXho5exd_hUY0w0KNBxuy-lI_ZbQNT1pB2beWshbx6N9L7R1pRIpC-RftZpOWjLUxrO0HzT"
access_token = get_access_token(auth_code)
print(f"Access Token: {access_token}")

    

Access Token: BQDMgBFJQkcUsZviwQQt2g9Ho3DX7I_ePF5NAVWI8wIbyqF6kjIxDUtpKTgT1xcdCGTOmNhdXmygQMliqtSIN8UYBsCVtSRf196UY0khqAia6aRCgc3ADSn68ZIhuX5mM3O79-8MTSaOGClgzXwSodBTICaqkIVQbmWEuDNLNLYdZ5Q5vkV7iMYtTr9j


In [112]:
ACCESS_TOKEN = access_token
BASE_URL = 'https://api.spotify.com/v1'

In [113]:
def get_top_tracks_for_range(access_token, time_range):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'limit': 50,
        'time_range': time_range
    }
    response = requests.get(f'{BASE_URL}/me/top/tracks', headers=headers, params=params)
    data = response.json()
    return data.get('items', [])

def get_artist_genres(artist_id, access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(f'{BASE_URL}/artists/{artist_id}', headers=headers)
    data = response.json()
    return data.get('genres', [])

def aggregate_genres_for_range(access_token, time_range):
    top_tracks = get_top_tracks_for_range(access_token, time_range)
    genre_count = {}
    
    for track in top_tracks:
        track_artists = track['artists']
        for artist in track_artists:
            artist_genres = get_artist_genres(artist['id'], access_token)
            for genre in artist_genres:
                if genre not in genre_count:
                    genre_count[genre] = 0
                genre_count[genre] += 1
    
    return genre_count

def get_top_genres_for_time_ranges(access_token):
    time_ranges = ['short_term', 'medium_term', 'long_term']
    all_genres = []

    for time_range in time_ranges:
        genres = aggregate_genres_for_range(access_token, time_range)
        all_genres.append({
            'time_range': time_range,
            'genres': genres
        })
    
    return all_genres

def convert_to_dataframe(genres_data):
    records = []
    for entry in genres_data:
        time_range = entry['time_range']
        for genre, count in entry['genres'].items():
            records.append({'time_range': time_range, 'genre': genre, 'count': count})
            
    df = pd.DataFrame(records)
    return df

In [116]:
genres_data = get_top_genres_for_time_ranges(ACCESS_TOKEN)
genre_df = convert_to_dataframe(genres_data)

{'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1h6Cn3P4NGzXbaXidqURXs'}, 'href': 'https://api.spotify.com/v1/artists/1h6Cn3P4NGzXbaXidqURXs', 'id': '1h6Cn3P4NGzXbaXidqURXs', 'name': 'Swedish House Mafia', 'type': 'artist', 'uri': 'spotify:artist:1h6Cn3P4NGzXbaXidqURXs'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'B

In [65]:
import requests
import pandas as pd

url = 'https://api.open-meteo.com/v1/forecast?latitude=41.0082&longitude=28.9784&hourly=temperature_2m&start_date=2024-12-01&end_date=2025-01-10'

response = requests.get(url)

if response.status_code == 200:
    data = response.json() 
    hourly_data = data['hourly']
    weather_df = pd.DataFrame(hourly_data)
    weather_df['time'] = pd.to_datetime(weather_df['time'])
    weather_df.set_index('time', inplace=True)
    daily_avg = weather_df.resample('D').mean()
    daily_avg.reset_index(inplace=True)    
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

In [123]:
daily_avg['time'] = pd.to_datetime(daily_avg['time'])
current_date = datetime.utcnow()

short_term_date = current_date - timedelta(days=5)
medium_term_date = current_date - timedelta(days=14)

def determine_time_range(row):
    if row['time'] >= short_term_date:
        return 'short_term'
    elif row['time'] >= medium_term_date:
        return 'medium_term'
    else:
        return 'long_term'

daily_avg['time_range'] = daily_avg.apply(determine_time_range, axis=1)

In [125]:
weather_data = daily_avg.groupby('time_range')['temperature_2m'].mean().reset_index()
weather_data.rename(columns={'temperature_2m': 'average_temperature'}, inplace=True)


In [126]:
weather_data

,time_range,average_temperature
0,long_term,10.102932
1,medium_term,7.850000
2,short_term,11.622500


In [129]:
all_data = pd.merge(genre_df, weather_data, on='time_range', how='left')